### COLMAP-NeRFStudio 간 좌표계 변환 실험 코드

In [ ]:

import numpy as np
import torch

# choice: Every transforms can be written as function or transformation matrix
T_opengl2opencv = np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
T_colmap2nerf = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, -1, 0, 0], [0, 0, 0, 1]]) # applied_transform

# convert between opencv and nerfstudio function version
def colmap_to_nerfstudio(c2w):
    # c2w: expected shape (4, 4)
    c2w[..., 0:3, 1:3] *= -1 # flip the y_c, z_c axis (2D OPENCV -> OPENGL)

    c2w = c2w[..., np.array([0, 2, 1, 3]), :] # exchange y, z (COLMAP -> NeRFStudio)
    c2w[..., 2, :] *= -1 # flip z axis (COLMAP -> NeRFStudio)
    return c2w

# convert between opencv and nerfstudio matrix version
def colmap_to_nerfstudio_matrix_ver(c2w):
    # c2w: expected shape (4, 4)

    T_opencv2colmap = c2w
    T_colmap_opencv = T_opencv2colmap
    
    T_opencv_opengl = T_opengl2opencv
    T_nerf_colmap = T_colmap2nerf    

    T_nerf_opengl = T_nerf_colmap @ T_colmap_opencv @ T_opencv_opengl
    T_opengl2nerf = T_nerf_opengl 
    c2w = T_opengl2nerf
    return c2w

# convert between opencv and opengl function version
def opengl_to_opencv(c2w):
    transform = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
    if isinstance(c2w, torch.Tensor):
        transform = torch.Tensor(transform).to(c2w)
    c2w[..., :3, :3] @= transform
    return c2w


In [ ]:
# [Test] coordinate transform (COLMAP -> NeRFStudio) evaluation
q_colmap2opencv = np.array([0.995759,0.000310773,0.0178788,-0.0902497]) 
t_colmap2opencv = np.array([4.34879,-1.51327,-0.0316503])
T_colmap2opencv = quaternion_to_transform_matrix(q_colmap2opencv, t_colmap2opencv) # T_cw
T_opencv2colmap = np.linalg.inv(T_colmap2opencv) # c2w
T_opengl2nerf = colmap_to_nerfstudio(T_opencv2colmap.copy())

# test 1
T_nerf_opengl = T_opengl2nerf
T_opengl_opencv = np.linalg.inv(T_opengl2opencv)
T_colmap_nerf = np.linalg.inv(T_colmap2nerf)
T_colmap_opencv_pred = T_colmap_nerf@T_nerf_opengl@T_opengl_opencv
T_opencv2colmap_pred = T_colmap_opencv_pred
print(np.allclose(T_opencv2colmap, T_opencv2colmap_pred, atol=1e-5))

# test 2
T_opengl2nerf_pred = colmap_to_nerfstudio_matrix_ver(T_opencv2colmap.copy())

print(np.allclose(T_opengl2nerf, T_opengl2nerf_pred, atol=1e-5))

True
True


### [Render] Transform cameras from transforms.json to camera_path.json

In [1]:

# Write Camera_path(opengl-NeRF to opengl-webgui)
import json
import numpy as np
import os
from R3DR.utils import dir_utils

def homogeneous(c2w):
    if len(c2w) == 3:        
        c2w = np.vstack([c2w, [0, 0, 0, 1]])
    return c2w

# 파일 경로
dataparser_transforms_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/ns_processed/nerfacto/2025-05-30_071924/dataparser_transforms.json"

workspace_dir = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2"
nerfstudio_transforms_path = os.path.join(workspace_dir, "new_correspondence/ns_processed/transforms.json")
webgui_camera_path = os.path.join(workspace_dir, "new_correspondence/ns_processed/camera_path.json")
config_path = os.path.join(workspace_dir, "new_correspondence/config/config.yaml")
fov = 40

config = dir_utils.load_config(config_path)
dir_utils.validate_config(config)

# dataparser_transforms.json 읽기
with open(dataparser_transforms_path, 'r') as file:
    dataparser = json.load(file)

# 변환 행렬과 스케일 가져오기(ok)
T_webgui_colmap = homogeneous(np.array(dataparser['transform'])) # (4, 4), dataparser_transform(T_webgui_colmap) = orient_pose_transform(T_webgui_nerf) @ applied_transform(T_nerf_colmap)
scale = dataparser['scale']
print(f"scale: {scale}")

# transforms.json 읽기
with open(nerfstudio_transforms_path, 'r') as file:
    transforms = json.load(file) # c2w

T_nerf_colmap = homogeneous(np.array(transforms['applied_transform'])) # (4, 4), applied_transform(T_nerf_colmap)
T_webgui_nerf =  T_webgui_colmap @ np.linalg.inv(T_nerf_colmap) # (4, 4) orient_pose_transform(T_webgui_nerf)

# 모든 프레임에 대해 변환 적용
for frame in transforms['frames']:

    T_nerf_opengl = np.array(frame['transform_matrix']) # (4, 4)
    

    # 변환 행렬 적용
    T_webgui_opengl = T_webgui_nerf @ T_nerf_opengl     
    T_webgui_opengl[:3, 3] *= scale
    frame['transform_matrix'] = T_webgui_opengl.flatten().tolist()
    

sorted_frames = sorted(transforms['frames'], key=lambda frame: frame['file_path'])

downscale_factor = int(config['nerf_settings']['ns_train_options']['dataparser_options']['downscale-factor'])

w, h = [int(v / downscale_factor) for v in (sorted_frames[0]['w'], sorted_frames[0]['h'])]
out = {
        'camera_type': 'perspective',
        'render_height': h,
        'render_width': w,
        'seconds': len(sorted_frames),
        'camera_path': [
                {'camera_to_world': frame['transform_matrix'], 'fov': fov, 'aspect': 1, 'file_path': frame['file_path']} for frame in sorted_frames
            ]
        }
# 변경된 내용을 transforms.json 파일에 다시 저장
with open(webgui_camera_path, 'w') as file:
    json.dump(out, file, indent=4)
print("Transformation applied and saved successfully.")


scale: 0.1889979446048152
Transformation applied and saved successfully.


In [ ]:
# Write camera_path(Opencv to ECEF)
import json
import numpy as np
from pyproj import Transformer
from plyfile import PlyData, PlyElement
from R3DR.utils.colmap_utils import qvec2rotmat
import time

T_opencv_opengl = np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

def homogeneous(c2w):
    if len(c2w) == 3:        
        c2w = np.vstack([c2w, [0, 0, 0, 1]])
    return c2w

def sim3_matrix(scale, rotation, translation):
    """
    Make a general sim3 matrix (scale -> rotation -> translation)
    """
    T = np.eye(4)
    T[:3, :3] = scale * rotation 
    T[:3, 3] = translation    
    return T

# 파일 경로
dataparser_transforms_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/ns_processed/nerfacto/2025-05-30_071924/dataparser_transforms.json"
ecef_transforms_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/ns_processed/colmap/sparse/geo-registration/ecef/sim3_transform.json"
webgui_camera_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/ns_processed/camera_path.json"

output_ecef_camera_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/ns_processed/ecef_camera_path.json"

# dataparser_transforms.json 읽기
with open(dataparser_transforms_path, 'r') as file:
    dataparser_tform = json.load(file)


T_webgui_colmap = homogeneous(np.array(dataparser_tform['transform'])) # (4, 4), dataparser_transform(T_webgui_colmap) = orient_pose_transform(T_webgui_nerf) @ applied_transform(T_nerf_colmap)
scale = dataparser_tform['scale']
print(scale)

# Sim(3) 변환 행렬 생성
# NeRFStudio Sim3 정의 상, R, t 먼저 적용 후 전체 scaling
T_webgui_colmap_sim3 = T_webgui_colmap.copy()
T_webgui_colmap_sim3[:3, :] *= scale
T_colmap_webgui_sim3 = np.linalg.inv(T_webgui_colmap_sim3)


# ecef_transforms.json 읽기
with open(ecef_transforms_path, 'r') as file:
    ecef_tform = file.read().strip()

scale, q0, q1, q2, q3, tx, ty, tz = map(np.float64, ecef_tform.split(" ")) # 13.708695938967331 0.44253099182081379 0.83353368302085051 0.31283289113402335 0.10734757925780858 -3047518.2350821388 4051223.3963172659 3857848.7263354594
rot = qvec2rotmat(np.array([q0, q1, q2, q3]))
trans = np.array([tx, ty, tz])

# Sim(3) 변환 행렬 생성 (scale first)
T_ecef_colmap_sim3 = sim3_matrix(scale, rot, trans)

T_ecef_webgui_sim3 = T_ecef_colmap_sim3 @ T_colmap_webgui_sim3

# camera_path.json 읽기
with open(webgui_camera_path, 'r') as file:
    camera_path_json = json.load(file)

out = {"frames": []}
for camera in camera_path_json['camera_path']:
    T_webgui_opengl = np.array(camera['camera_to_world']).reshape(4, 4)
    file_path = camera['file_path']

    T_ecef_opencv = T_ecef_webgui_sim3 @ T_webgui_opengl @ np.linalg.inv(T_opencv_opengl)    

    # retain orthonormal property of rotation
    R_wc = T_ecef_opencv[:3, :3] 
    U, _, V_t = np.linalg.svd(R_wc)
    R_wc_norm = U @ V_t
    T_ecef_opencv[:3, :3] = R_wc_norm

    out["frames"].append({'file_path': file_path, 'camera_to_world': T_ecef_opencv.tolist()})

# 변경된 내용을 transforms.json 파일에 저장
with open(output_ecef_camera_path, 'w') as file:
    json.dump(out, file, indent=4)
print("Transformation applied and saved successfully.")


Transformation applied and saved successfully.


### [Geo-registration] Transform pcd/mesh from NeRFStudio to ECEF/UTM

In [ ]:
# NeRFStudio(WebGUI) pcd -> ECEF/UTM Geo-registered pcd
import json
import numpy as np
from pyproj import Transformer
from R3DR.utils.colmap_utils import qvec2rotmat
import time
import open3d as o3d
import os

def homogeneous(c2w):
    if len(c2w) == 3:        
        c2w = np.vstack([c2w, [0, 0, 0, 1]])
    return c2w

def normalize_coordinates(coords):
    """
    좌표 배열에서 각 축의 평균을 계산한 후, 이를 원래 좌표에서 뺀 값을 반환합니다.
    
    Parameters:
        coords (numpy.ndarray): (n, 3) 형태의 좌표 배열.
    
    Returns:
        numpy.ndarray: 평균이 제거된 (n, 3) 형태의 좌표 배열.
    """
    mean_coords = np.mean(coords, axis=0)  # 각 축(x, y, z)의 평균 계산
    normalized_coords = coords - mean_coords  # 평균을 빼서 원점 기준 정규화
    return normalized_coords, mean_coords
    
def sim3_matrix(scale, rotation, translation):
    """
    Make a general sim3 matrix (scale -> rotation -> translation)
    """
    T = np.eye(4)
    T[:3, :3] = scale * rotation 
    T[:3, 3] = translation    
    return T

def ecef_to_utm_transformer(zone_number: int, zone_letter: str):
    """Creates a Transformer for ECEF to UTM conversion."""
    return Transformer.from_crs("EPSG:4978", f"+proj=utm +zone={zone_number} +{zone_letter} +datum=WGS84")

def ecef_to_utm_points(points_ecef: np.ndarray , transformer: Transformer) -> np.ndarray:
    """
    Converts ECEF points to UTM coordinates.
    
    points_ecef : (N, 3)
    """

    x, y, z = points_ecef[:, 0], points_ecef[:, 1], points_ecef[:, 2]
    e, n, alt = transformer.transform(x, y, z)
    points_utm = np.stack([e, n, alt], axis = -1) 
    
    return np.array(points_utm)

def ecef_to_central_korea_transformer():
    """Creates a Transformer for ECEF to EPSG:5186 (중부원점) conversion."""
    return Transformer.from_crs("EPSG:4978", "EPSG:5186", always_xy=True)

def ecef_to_central_korea_points(points_ecef: np.ndarray , transformer: Transformer) -> np.ndarray:
    """
    Converts ECEF points to 중부원점 (EPSG:5186) coordinates.
    
    points_ecef : (N, 3)
    """
    x, y, z = points_ecef[:, 0], points_ecef[:, 1], points_ecef[:, 2]
    e, n, alt = transformer.transform(x, y, z)
    return np.stack([e, n, alt], axis=-1)
    
##############################################################################################################################################
# User Input
##############################################################################################################################################
workspace_dir = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial"
dataparser_transforms_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/ns_processed/nerfacto/2025-05-30_071924/dataparser_transforms.json"
##############################################################################################################################################
ecef_transforms_path = os.path.join(workspace_dir, "ns_processed/colmap/sparse/geo-registration/ecef/sim3_transform.json")

ply_path = os.path.join(workspace_dir, "nerf_output/exports/pcd/point_cloud.ply")

output_dir = os.path.join(workspace_dir, "nerf_output/exports/pcd/original")

# 디렉토리 생성 (존재하지 않으면)
os.makedirs(output_dir, exist_ok=True)

# ECEF Geo-registered PLY 저장 경로
output_ecef_ply_path = os.path.join(output_dir, "point_cloud_ecef.ply")

# UTM Geo-registered PLY 저장 경로
output_utm_ply_path = os.path.join(output_dir, "point_cloud_utm.ply")

# CK Geo-registered PLY 저장 경로
output_ck_ply_path = os.path.join(output_dir, "point_cloud_ck.ply")

# output_ck_translation_path = os.path.join(workspace_dir, "translation_ck.json")

# Start timing
start_time = time.time()

# dataparser_transforms.json 읽기
with open(dataparser_transforms_path, 'r') as file:
    dataparser_tform = json.load(file)


T_webgui_colmap = homogeneous(np.array(dataparser_tform['transform'])) # (4, 4), dataparser_transform(T_webgui_colmap) = orient_pose_transform(T_webgui_nerf) @ applied_transform(T_nerf_colmap)
scale = dataparser_tform['scale']


# Sim(3) 변환 행렬 생성
# NeRFStudio Sim3 정의 상, R, t 먼저 적용 후 전체 scaling
T_webgui_colmap_sim3 = T_webgui_colmap.copy()
T_webgui_colmap_sim3[:3, :] *= scale
T_colmap_webgui_sim3 = np.linalg.inv(T_webgui_colmap_sim3)

print(f"Loaded dataparser_transform in {time.time() - start_time:.2f} seconds")

start_time = time.time()

# ecef_transforms.json 읽기
with open(ecef_transforms_path, 'r') as file:
    ecef_tform = file.read().strip()

scale, q0, q1, q2, q3, tx, ty, tz = map(np.float64, ecef_tform.split(" ")) # 13.708695938967331 0.44253099182081379 0.83353368302085051 0.31283289113402335 0.10734757925780858 -3047518.2350821388 4051223.3963172659 3857848.7263354594
rot = qvec2rotmat(np.array([q0, q1, q2, q3]))
trans = np.array([tx, ty, tz])

# Sim(3) 변환 행렬 생성 (scale first)
T_ecef_colmap_sim3 = sim3_matrix(scale, rot, trans)

T_ecef_webgui_sim3 = T_ecef_colmap_sim3 @ T_colmap_webgui_sim3

print(f"Loaded ECEF transform in {time.time() - start_time:.2f} seconds")

start_time = time.time()

# .ply 파일 읽기
pcd = o3d.io.read_point_cloud(ply_path)
points = np.asarray(pcd.points)  # 포인트 데이터 가져오기
colors = np.asarray(pcd.colors) if pcd.has_colors() else None
normals = np.asarray(pcd.normals) if pcd.has_normals() else None

print(f"Loaded PLY data in {time.time() - start_time:.2f} seconds")

start_time = time.time()
# ECEF 변환
coords = np.hstack([points, np.ones((points.shape[0], 1))])  # (N, 4)
assert(coords.dtype == np.float64)

coords_ecef = (T_ecef_webgui_sim3[:3, :] @ coords.T).T  # ECEF 변환 좌표 (x', y', z')

print(f"Converted to ECEF in {time.time() - start_time:.2f} seconds")
start_time = time.time()

# Convert ECEF to UTM
transformer = ecef_to_utm_transformer(zone_number=52, zone_letter="north")
coords_utm = ecef_to_utm_points(coords_ecef, transformer)

print(f"Converted to UTM in {time.time() - start_time:.2f} seconds")

# 중부원점 변환
transformer_central_korea = ecef_to_central_korea_transformer()
coords_ck = ecef_to_central_korea_points(coords_ecef, transformer_central_korea)
print(f"Converted to central korea in {time.time() - start_time:.2f} seconds")

# ECEF PCD 객체 생성
ecef_pcd = o3d.geometry.PointCloud()
ecef_pcd.points = o3d.utility.Vector3dVector(coords_ecef)  # ECEF 좌표 (x, y, z)
if colors is not None:
    ecef_pcd.colors = o3d.utility.Vector3dVector(colors)

if normals is not None:
    ecef_pcd.normals = o3d.utility.Vector3dVector(normals)


# UTM PCD 객체 생성
utm_pcd = o3d.geometry.PointCloud()
utm_pcd.points = o3d.utility.Vector3dVector(coords_utm)  # UTM 좌표 (x, y, z)
if colors is not None:
    utm_pcd.colors = o3d.utility.Vector3dVector(colors)

if normals is not None:
    utm_pcd.normals = o3d.utility.Vector3dVector(normals)


# 중부원점 PCD 객체 생성
# coords_ck, ck_global_translation = normalize_coordinates(coords_ck)

ck_pcd = o3d.geometry.PointCloud()
ck_pcd.points = o3d.utility.Vector3dVector(coords_ck)
if colors is not None:
    ck_pcd.colors = o3d.utility.Vector3dVector(colors)
if normals is not None:
    ck_pcd.normals = o3d.utility.Vector3dVector(normals)

# mean_dict = {"global_translation": ck_global_translation.tolist()}

# with open(output_ck_translation_path, "w") as f:
#     json.dump(mean_dict, f, indent=4)  # JSON 파일로 저장 (들여쓰기 포함)

# ECEF .ply 파일로 저장
o3d.io.write_point_cloud(output_ecef_ply_path, ecef_pcd)
print(f"ECEF Transformed point cloud saved to: {output_ecef_ply_path}")

# UTM .ply 파일로 저장
o3d.io.write_point_cloud(output_utm_ply_path, utm_pcd)
print(f"UTM Transformed point cloud saved to: {output_utm_ply_path}")

# 중부원점 PLY 저장
o3d.io.write_point_cloud(output_ck_ply_path, ck_pcd)
print(f"중부원점 (EPSG:5186) Transformed point cloud saved to: {output_ck_ply_path}")


Loaded dataparser_transform in 0.00 seconds
Loaded ECEF transform in 0.00 seconds
Loaded PLY data in 2.01 seconds
Converted to ECEF in 0.17 seconds
Converted to UTM in 3.07 seconds
Converted to central korea in 6.07 seconds
ECEF Transformed point cloud saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/exports/pcd/original/point_cloud_ecef.ply
UTM Transformed point cloud saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/exports/pcd/original/point_cloud_utm.ply
중부원점 (EPSG:5186) Transformed point cloud saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/exports/pcd/original/point_cloud_ck.pcd


In [21]:
# UTM Geo-registered pcd(Filtered PCD) -> NeRFStudio(WebGUI) pcd(For mesh generation and filtered central pcd)
import json
import numpy as np
from pyproj import Transformer, CRS
import time
import open3d as o3d
from R3DR.utils.colmap_utils import qvec2rotmat

def homogeneous(c2w):
    if len(c2w) == 3:        
        c2w = np.vstack([c2w, [0, 0, 0, 1]])
    return c2w

def sim3_matrix(scale, rotation, translation):
    """
    Make a general sim3 matrix (scale -> rotation -> translation)
    """
    T = np.eye(4)
    T[:3, :3] = scale * rotation 
    T[:3, 3] = translation    
    return T

def normalize_coordinates(coords):
    """
    좌표 배열에서 각 축의 평균을 계산한 후, 이를 원래 좌표에서 뺀 값을 반환합니다.
    
    Parameters:
        coords (numpy.ndarray): (n, 3) 형태의 좌표 배열.
    
    Returns:
        numpy.ndarray: 평균이 제거된 (n, 3) 형태의 좌표 배열.
    """
    mean_coords = np.mean(coords, axis=0)  # 각 축(x, y, z)의 평균 계산
    normalized_coords = coords - mean_coords  # 평균을 빼서 원점 기준 정규화
    return normalized_coords, mean_coords

def ecef_to_central_korea_transformer():
    """Creates a Transformer for ECEF to EPSG:5186 (중부원점) conversion."""
    return Transformer.from_crs("EPSG:4978", "EPSG:5186", always_xy=True)

def ecef_to_central_korea_points(points_ecef: np.ndarray , transformer: Transformer) -> np.ndarray:
    """
    Converts ECEF points to 중부원점 (EPSG:5186) coordinates.
    
    points_ecef : (N, 3)
    """
    x, y, z = points_ecef[:, 0], points_ecef[:, 1], points_ecef[:, 2]
    e, n, alt = transformer.transform(x, y, z)
    return np.stack([e, n, alt], axis=-1)

def utm_to_ecef_transformer(zone_number: int, zone_letter: str):
    """Creates a Transformer for UTM to ECEF conversion."""
    south = " +south" if zone_letter == "S" else ""  # 남반구 설정
    utm_crs = CRS.from_proj4(f"+proj=utm +zone={zone_number}{south} +datum=WGS84 +units=m +no_defs")
    
    transformer_utm_to_ecef = Transformer.from_crs(utm_crs, "EPSG:4978", always_xy=True)
    return transformer_utm_to_ecef  # 올바르게 변환기를 반환

def utm_to_ecef_points(points_utm: np.ndarray, transformer: Transformer) -> np.ndarray:
    """
    UTM (easting, northing, altitude) 좌표를 ECEF 좌표로 변환합니다.
    
    points_utm : (N, 3)
    transformer : UTM->ECEF 변환을 위한 pyproj Transformer
    """
    # UTM 좌표: (easting, northing, altitude)
    east, north, alt = points_utm[:, 0], points_utm[:, 1], points_utm[:, 2]
    # Transformer.transform의 입력 순서는 (x, y, z) 즉, (east, north, alt)
    x, y, z = transformer.transform(east, north, alt)
    points_ecef = np.stack([x, y, z], axis=-1)
    return points_ecef

##############################################################################################################################################
# User Input
##############################################################################################################################################
workspace_dir = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial"
dataparser_transforms_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/ns_processed/nerfacto/2025-05-30_071924/dataparser_transforms.json"
##############################################################################################################################################
ecef_transforms_path = os.path.join(workspace_dir, "ns_processed/colmap/sparse/geo-registration/ecef/sim3_transform.json")

utm_ply_path = os.path.join(workspace_dir, "nerf_output/exports/pcd/filtered/filtered_utm.ply")

output_webgui_ply_path = os.path.join(workspace_dir, "nerf_output/exports/pcd/filtered/filtered_webgui.pcd")

output_ck_ply_path = os.path.join(workspace_dir, "nerf_output/exports/pcd/filtered/filtered_ck.pcd")
output_ck_translation_path = os.path.join(workspace_dir, "nerf_output/exports/pcd/filtered/translation_ck.json")

# dataparser_transforms.json 읽기
with open(dataparser_transforms_path, 'r') as file:
    dataparser_tform = json.load(file)

T_webgui_colmap = homogeneous(np.array(dataparser_tform['transform'])) # (4, 4), dataparser_transform(T_webgui_colmap) = orient_pose_transform(T_webgui_nerf) @ applied_transform(T_nerf_colmap)
scale = dataparser_tform['scale']


# Sim(3) 변환 행렬 생성
# NeRFStudio Sim3 정의 상, R, t 먼저 적용 후 전체 scaling
T_webgui_colmap_sim3 = T_webgui_colmap.copy()
T_webgui_colmap_sim3[:3, :] *= scale
T_colmap_webgui_sim3 = np.linalg.inv(T_webgui_colmap_sim3)

start_time = time.time()

# ecef_transforms.json 읽기
with open(ecef_transforms_path, 'r') as file:
    ecef_tform = file.read().strip()

scale, q0, q1, q2, q3, tx, ty, tz = map(np.float64, ecef_tform.split(" ")) # 13.708695938967331 0.44253099182081379 0.83353368302085051 0.31283289113402335 0.10734757925780858 -3047518.2350821388 4051223.3963172659 3857848.7263354594
rot = qvec2rotmat(np.array([q0, q1, q2, q3]))
trans = np.array([tx, ty, tz])

# Sim(3) 변환 행렬 생성 (scale first)
T_ecef_colmap_sim3 = sim3_matrix(scale, rot, trans)

T_ecef_webgui_sim3 = T_ecef_colmap_sim3 @ T_colmap_webgui_sim3
T_webgui_ecef_sim3 = np.linalg.inv(T_ecef_webgui_sim3)

print(f"Loaded ECEF transform in {time.time() - start_time:.2f} seconds")


start_time = time.time()

# .ply 파일 읽기
pcd = o3d.io.read_point_cloud(utm_ply_path)
coords_utm = np.asarray(pcd.points)  # 포인트 데이터 가져오기
assert(coords_utm.dtype == np.float64)
colors = np.asarray(pcd.colors) if pcd.has_colors() else None
normals = np.asarray(pcd.normals) if pcd.has_normals() else None

print(f"Loaded PLY data in {time.time() - start_time:.2f} seconds")

start_time = time.time()

# Convert UTM to ECEF
transformer = utm_to_ecef_transformer(zone_number=52, zone_letter="north")
coords_ecef = utm_to_ecef_points(coords_utm, transformer)
coords_ecef = np.hstack([coords_ecef, np.ones((coords_ecef.shape[0], 1))])  # (N, 4)
print(f"Converted to UTM in {time.time() - start_time:.2f} seconds")

# 중부원점 변환
start_time = time.time()
transformer_central_korea = ecef_to_central_korea_transformer()
coords_ck = ecef_to_central_korea_points(coords_ecef, transformer_central_korea)
print(f"Converted to central korea in {time.time() - start_time:.2f} seconds")

start_time = time.time()

# Convert ECEF to WebGUI
coords_webgui = (T_webgui_ecef_sim3[:3, :] @ coords_ecef.T).T

print(f"Converted to NeRFStudio(WebGUI) in {time.time() - start_time:.2f} seconds")


# WebGUI PCD 객체 생성
webgui_pcd = o3d.geometry.PointCloud()
webgui_pcd.points = o3d.utility.Vector3dVector(coords_webgui)  # ECEF 좌표 (x, y, z)
if colors is not None:
    webgui_pcd.colors = o3d.utility.Vector3dVector(colors)

if normals is not None:
    webgui_pcd.normals = o3d.utility.Vector3dVector(normals)


# 중부원점 PCD 객체 생성
ck_pcd = o3d.geometry.PointCloud()
coords_ck, ck_global_translation = normalize_coordinates(coords_ck)
ck_pcd.points = o3d.utility.Vector3dVector(coords_ck)
if colors is not None:
    ck_pcd.colors = o3d.utility.Vector3dVector(colors)
if normals is not None:
    ck_pcd.normals = o3d.utility.Vector3dVector(normals)


# 중부원점 PLY 저장
o3d.io.write_point_cloud(output_ck_ply_path, ck_pcd)
print(f"중부원점 (EPSG:5186) Transformed point cloud saved to: {output_ck_ply_path}")

mean_dict = {"global_translation": ck_global_translation.tolist()}

with open(output_ck_translation_path, "w") as f:
    json.dump(mean_dict, f, indent=4)  # JSON 파일로 저장 (들여쓰기 포함)


# ECEF .ply 파일로 저장
o3d.io.write_point_cloud(output_webgui_ply_path, webgui_pcd)
print(f"webgui Transformed point cloud saved to: {output_webgui_ply_path}")


Loaded ECEF transform in 0.00 seconds
Loaded PLY data in 1.71 seconds
Converted to UTM in 2.94 seconds
Converted to central korea in 2.62 seconds
Converted to NeRFStudio(WebGUI) in 0.03 seconds
중부원점 (EPSG:5186) Transformed point cloud saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/exports/pcd/filtered/filtered_ck.pcd
webgui Transformed point cloud saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/exports/pcd/filtered/filtered_webgui.pcd


In [6]:
# 3D NeRFStudio mesh -> ECEF/UTM Translation

import json
import numpy as np
from pyproj import Transformer
from R3DR.utils.colmap_utils import qvec2rotmat
import time
import open3d as o3d
import copy
import os

def homogeneous(c2w):
    if len(c2w) == 3:        
        c2w = np.vstack([c2w, [0, 0, 0, 1]])
    return c2w

def sim3_matrix(scale, rotation, translation):
    """
    Make a general sim3 matrix (scale -> rotation -> translation)
    """
    T = np.eye(4)
    T[:3, :3] = scale * rotation 
    T[:3, 3] = translation    
    return T

def ecef_to_utm_transformer(zone_number: int, zone_letter: str):
    """Creates a Transformer for ECEF to UTM conversion."""
    return Transformer.from_crs("EPSG:4978", f"+proj=utm +zone={zone_number} +{zone_letter} +datum=WGS84")

def ecef_to_utm_points(points_ecef: np.ndarray , transformer: Transformer) -> np.ndarray:
    """
    Converts ECEF points to UTM coordinates.
    
    points_ecef : (N, 3)
    """

    x, y, z = points_ecef[:, 0], points_ecef[:, 1], points_ecef[:, 2]
    e, n, alt = transformer.transform(x, y, z)
    points_utm = np.stack([e, n, alt], axis = -1) 
    
    return points_utm

def ecef_to_central_korea_transformer():
    """Creates a Transformer for ECEF to EPSG:5186 (중부원점) conversion."""
    return Transformer.from_crs("EPSG:4978", "EPSG:5186", always_xy=True)

def ecef_to_central_korea_points(points_ecef: np.ndarray , transformer: Transformer) -> np.ndarray:
    """
    Converts ECEF points to 중부원점 (EPSG:5186) coordinates.
    
    points_ecef : (N, 3)
    """
    x, y, z = points_ecef[:, 0], points_ecef[:, 1], points_ecef[:, 2]
    e, n, alt = transformer.transform(x, y, z)
    return np.stack([e, n, alt], axis=-1)

def ecef_to_enu_rotation_matrix(lat_deg, lon_deg):
    """
    위도/경도로 정의된 ENU 기준 좌표계에 대한 ECEF → ENU 회전 행렬 생성
    """
    lat = np.radians(lat_deg)
    lon = np.radians(lon_deg)

    R = np.array([
        [-np.sin(lon),              np.cos(lon),              0],
        [-np.sin(lat)*np.cos(lon), -np.sin(lat)*np.sin(lon), np.cos(lat)],
        [np.cos(lat)*np.cos(lon),  np.cos(lat)*np.sin(lon),  np.sin(lat)]
    ])
    return R  # 3x3 matrix


def ecef_to_latlonalt(x, y, z):
    transformer = Transformer.from_crs("epsg:4978", "epsg:4326", always_xy=True)  # ECEF → WGS84
    lon, lat, alt = transformer.transform(x, y, z)
    return lat, lon, alt

def normalize_coordinates(coords):
    """
    좌표 배열에서 각 축의 평균을 계산한 후, 이를 원래 좌표에서 뺀 값을 반환합니다.
    
    Parameters:
        coords (numpy.ndarray): (n, 3) 형태의 좌표 배열.
    
    Returns:
        numpy.ndarray: 평균이 제거된 (n, 3) 형태의 좌표 배열.
    """
    mean_coords = np.mean(coords, axis=0)  # 각 축(x, y, z)의 평균 계산
    normalized_coords = coords - mean_coords  # 평균을 빼서 원점 기준 정규화
    return normalized_coords, mean_coords


# Open3D의 mesh load시, Vertex 중복 문제를 해소하기 위함(Texture mapping을 위한 코드)
def remove_duplicate_vertices(mesh):
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)
    triangle_uvs = np.asarray(mesh.triangle_uvs)
    vertex_normals = np.asarray(mesh.vertex_normals)
    
    # 중복 제거를 위한 dict (순서 보존) np.unique는 자동 sort되어 순서기반 mapping이 깨짐
    unique_vertex_map = {}  
    new_vertices = []
    inverse_indices = np.zeros(len(vertices), dtype=int)

    for i, v in enumerate(vertices):
        v_tuple = tuple(v)  # 리스트를 튜플로 변환 (딕셔너리 key로 사용)
        if v_tuple not in unique_vertex_map:
            unique_vertex_map[v_tuple] = len(new_vertices)
            new_vertices.append(v)
        inverse_indices[i] = unique_vertex_map[v_tuple]
    print(f"Maximum in inverse_indices: {np.max(inverse_indices)}")
    new_vertices = np.array(new_vertices)
    new_triangles = inverse_indices[triangles]
    assert triangles.shape == new_triangles.shape # 실제로 중복된 면은 없다.

    new_triangle_uvs = triangle_uvs.copy()
    new_triangle_uvs[:, 1] = 1.0 - new_triangle_uvs[:, 1]  # Y축 뒤집기 (Open3D와 다른 mesh 소프트웨어 간 UV 정의 차이 보정)

    
    print(f"Original Vertex Shape: {vertices.shape}")
    print(f"New Vertex Shape: {new_vertices.shape}")

    # 정점 병합 후, 법선 평균 처리
    new_normals = np.zeros_like(new_vertices, dtype=np.float64)
    normal_counts = np.zeros(len(new_vertices), dtype=int)

    for old_idx, new_idx in enumerate(inverse_indices):
        new_normals[new_idx] += vertex_normals[old_idx]
        normal_counts[new_idx] += 1

    new_normals /= normal_counts[:, None]  # 정규화        
    
    # Open3D mesh 갱신
    new_mesh = o3d.geometry.TriangleMesh()
    new_mesh.vertices = o3d.utility.Vector3dVector(new_vertices)
    new_mesh.triangles = o3d.utility.Vector3iVector(new_triangles)
    new_mesh.triangle_uvs = o3d.utility.Vector2dVector(new_triangle_uvs)
    new_mesh.vertex_normals = o3d.utility.Vector3dVector(new_normals)
    
    return new_mesh

# -------------------------------
# 📌 .OBJ 메쉬 파일 변환 코드
# -------------------------------
def transform_mesh_obj(input_obj_path, texture_path, output_path, T_ecef_webgui_sim3, zone_number=52, zone_letter="north"):
    start_time = time.time()
    
    output_ecef_obj_path = os.path.join(output_path, "mesh_ecef.obj")
    output_utm_obj_path = os.path.join(output_path, "mesh_utm.obj")
    output_ck_obj_path = os.path.join(output_path, "mesh_ck.obj")
    output_ecef_translation_path = os.path.join(output_path, "translation_ecef.json")
    output_utm_translation_path = os.path.join(output_path, "translation_utm.json")
    output_ck_translation_path = os.path.join(output_path, "translation_ck.json")

    # 1️⃣ .OBJ 파일 로드
    mesh = o3d.io.read_triangle_mesh(input_obj_path) 
    
    new_mesh = remove_duplicate_vertices(mesh)   

    image = o3d.io.read_image(texture_path)  # 텍스처 로드
    new_mesh.textures = [o3d.geometry.Image(image)]  # 텍스처 적용

    vertices = np.asarray(new_mesh.vertices)    
    normals = np.asarray(new_mesh.vertex_normals)
    
    print(f"Loaded OBJ mesh in {time.time() - start_time:.2f} seconds")

    # 2️⃣ ECEF 변환 적용
    start_time = time.time()
    coords = np.hstack([vertices, np.ones((vertices.shape[0], 1))])  # (N, 4)
    normals_homo = np.hstack([normals, np.zeros((normals.shape[0], 1))])  # (N, 4)
    assert coords.dtype == np.float64
    
    coords_ecef = (T_ecef_webgui_sim3[:3, :] @ coords.T).T  # ECEF 좌표 변환 (N, 3)
    normals_ecef = (T_ecef_webgui_sim3[:3, :] @ normals_homo.T).T  # ECEF 좌표 변환 (N, 3)
    normals_ecef /=  np.linalg.norm(normals_ecef)

    latlonalts = []
    for x, y, z in coords_ecef:
        latlonalts.append(ecef_to_latlonalt(x, y, z))    
    latlonalts = np.asarray(latlonalts) # (N, 3)

    normals_enu = []
    for (lat, lon, alt), normal_ecef in zip(latlonalts, normals_ecef):
        R_ecef2enu = ecef_to_enu_rotation_matrix(lat, lon)
        normals_enu.append(R_ecef2enu @ normal_ecef)

    print(f"Converted to ECEF in {time.time() - start_time:.2f} seconds")

    # 3️⃣ ECEF → UTM 변환
    start_time = time.time()
    transformer = ecef_to_utm_transformer(zone_number, zone_letter)
    coords_utm = ecef_to_utm_points(coords_ecef, transformer)

    print(f"Converted to UTM in {time.time() - start_time:.2f} seconds")

    # 중부원점 변환
    start_time = time.time()
    transformer_central_korea = ecef_to_central_korea_transformer()
    coords_ck = ecef_to_central_korea_points(coords_ecef, transformer_central_korea)
    print(f"Converted to central korea in {time.time() - start_time:.2f} seconds")

    # 4️⃣ 변환된 좌표를 Open3D Mesh로 저장
    start_time = time.time()
    
    # ECEF 변환 후 저장
    ecef_mesh = copy.deepcopy(new_mesh)
    coords_ecef, ecef_global_translation = normalize_coordinates(coords_ecef)
    ecef_mesh.vertices = o3d.utility.Vector3dVector(coords_ecef)    
    ecef_mesh.vertex_normals = o3d.utility.Vector3dVector(normals_ecef)    
    o3d.io.write_triangle_mesh(output_ecef_obj_path, ecef_mesh, 
        write_ascii=True,
        compressed=False,      # 압축 없이 저장 (정밀도 유지)
        write_vertex_normals=True,  # 법선 포함
        write_vertex_colors=False,  # 색상 포함 X (필요한 경우 True)
        write_triangle_uvs=True     # UV 좌표 강제 포함
    )
    mean_dict = {"global_translation": ecef_global_translation.tolist()}

    with open(output_ecef_translation_path, "w") as f:
        json.dump(mean_dict, f, indent=4)  # JSON 파일로 저장 (들여쓰기 포함)
    
    print(f"ECEF Transformed mesh saved to: {output_ecef_obj_path}")

    # UTM 변환 후 저장
    utm_mesh = copy.deepcopy(new_mesh)    
    coords_utm, utm_global_translation = normalize_coordinates(coords_utm)
    utm_mesh.vertices = o3d.utility.Vector3dVector(coords_utm)        
    utm_mesh.vertex_normals = o3d.utility.Vector3dVector(normals_enu)
    o3d.io.write_triangle_mesh(output_utm_obj_path, utm_mesh, 
        write_ascii=True,
        compressed=False,      # 압축 없이 저장 (정밀도 유지)
        write_vertex_normals=True,  # 법선 포함
        write_vertex_colors=False,  # 색상 포함 X (필요한 경우 True)
        write_triangle_uvs=True     # UV 좌표 강제 포함
    )
    mean_dict = {"global_translation": utm_global_translation.tolist()}

    with open(output_utm_translation_path, "w") as f:
        json.dump(mean_dict, f, indent=4)  # JSON 파일로 저장 (들여쓰기 포함)
        
    print(f"UTM Transformed mesh saved to: {output_utm_obj_path}")

    # CK 변환 후 저장
    ck_mesh = copy.deepcopy(new_mesh)    
    coords_ck, ck_global_translation = normalize_coordinates(coords_ck)
    ck_mesh.vertices = o3d.utility.Vector3dVector(coords_ck)        
    ck_mesh.vertex_normals = o3d.utility.Vector3dVector(normals_enu)
    o3d.io.write_triangle_mesh(output_ck_obj_path, ck_mesh, 
        write_ascii=True,
        compressed=False,      # 압축 없이 저장 (정밀도 유지)
        write_vertex_normals=True,  # 법선 포함
        write_vertex_colors=False,  # 색상 포함 X (필요한 경우 True)
        write_triangle_uvs=True     # UV 좌표 강제 포함
    )
    mean_dict = {"global_translation": ck_global_translation.tolist()}

    with open(output_ck_translation_path, "w") as f:
        json.dump(mean_dict, f, indent=4)  # JSON 파일로 저장 (들여쓰기 포함)
        
    print(f"CK Transformed mesh saved to: {output_ck_obj_path}")

    print(f"Saved transformed meshes in {time.time() - start_time:.2f} seconds")


dataparser_transforms_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/ns_processed/nerfacto/2025-05-08_071846/dataparser_transforms.json"
ecef_transforms_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/ns_processed/colmap/sparse/geo-registration/ecef/sim3_transform.json"

obj_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/exports/3dmesh/mesh.obj"

texture_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/exports/3dmesh/material_0.png"  # 텍스처 파일 경로

output_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/exports/3dmesh/gps"
os.makedirs(output_path, exist_ok=True)

# Start timing
start_time = time.time()

# dataparser_transforms.json 읽기
with open(dataparser_transforms_path, 'r') as file:
    dataparser_tform = json.load(file)


T_webgui_colmap = homogeneous(np.array(dataparser_tform['transform'])) # (4, 4), dataparser_transform(T_webgui_colmap) = orient_pose_transform(T_webgui_nerf) @ applied_transform(T_nerf_colmap)
scale = dataparser_tform['scale']


# Sim(3) 변환 행렬 생성
# NeRFStudio Sim3 정의 상, R, t 먼저 적용 후 전체 scaling
T_webgui_colmap_sim3 = T_webgui_colmap.copy()
T_webgui_colmap_sim3[:3, :] *= scale
T_colmap_webgui_sim3 = np.linalg.inv(T_webgui_colmap_sim3)

print(f"Loaded dataparser_transform in {time.time() - start_time:.2f} seconds")

start_time = time.time()

# ecef_transforms.json 읽기
with open(ecef_transforms_path, 'r') as file:
    ecef_tform = file.read().strip()

scale, q0, q1, q2, q3, tx, ty, tz = map(np.float64, ecef_tform.split(" ")) # 13.708695938967331 0.44253099182081379 0.83353368302085051 0.31283289113402335 0.10734757925780858 -3047518.2350821388 4051223.3963172659 3857848.7263354594
rot = qvec2rotmat(np.array([q0, q1, q2, q3]))
trans = np.array([tx, ty, tz])

# Sim(3) 변환 행렬 생성 (scale first)
T_ecef_colmap_sim3 = sim3_matrix(scale, rot, trans)

T_ecef_webgui_sim3 = T_ecef_colmap_sim3 @ T_colmap_webgui_sim3

print(f"Loaded ECEF transform in {time.time() - start_time:.2f} seconds")

start_time = time.time()

transform_mesh_obj(obj_path, texture_path, output_path, T_ecef_webgui_sim3)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loaded dataparser_transform in 0.01 seconds
Loaded ECEF transform in 0.00 seconds
Maximum in inverse_indices: 208191
Original Vertex Shape: (342598, 3)
New Vertex Shape: (208192, 3)
Loaded OBJ mesh in 10.77 seconds
Converted to ECEF in 2099.12 seconds
Converted to UTM in 0.12 seconds
Converted to central korea in 0.14 seconds
ECEF Transformed mesh saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/exports/3dmesh/gps/mesh_ecef.obj
UTM Transformed mesh saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/exports/3dmesh/gps/mesh_utm.obj
CK Transformed mesh saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/in

### PCD Post-Process

In [14]:
import open3d as o3d
import numpy as np
import os

# .ply 파일 로드
workspace_dir = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial"
input_ply_path = os.path.join(workspace_dir,'nerf_output/exports/pcd/original/point_cloud_utm.ply')

# 저장할 .ply 파일 경로
output_dir = os.path.join(workspace_dir,'nerf_output/exports/pcd/filtered')
# 디렉토리 생성 (존재하지 않으면)
os.makedirs(output_dir, exist_ok=True)

output_ply_path = os.path.join(output_dir, 'filtered_utm.ply')

pcd = o3d.io.read_point_cloud(input_ply_path)
print(f"Loaded {len(pcd.points)} points from {input_ply_path}")

Loaded 9998607 points from /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/exports/pcd/original/point_cloud_utm.ply


In [15]:
import numpy as np
from shapely.geometry import Point, Polygon
from pyproj import Transformer

# ====== 함수 정의 ======
def filter_points_in_polygon(point_cloud_xy, polygon_xy):
    polygon = Polygon(polygon_xy)
    ind = np.array([polygon.contains(Point(x, y)) for x, y in point_cloud_xy])
    return ind

def ecef_to_utm_transformer(zone_number: int, zone_letter: str):
    return Transformer.from_crs("EPSG:4978", f"+proj=utm +zone={zone_number} +{zone_letter} +datum=WGS84")

def ecef_to_utm_points(points_ecef: np.ndarray, transformer: Transformer) -> np.ndarray:
    x, y, z = points_ecef[:, 0], points_ecef[:, 1], points_ecef[:, 2]
    e, n, alt = transformer.transform(x, y, z)
    return np.stack([e, n, alt], axis=-1)

def geodetic_to_ecef(lat, lon, alt=0.0):
    a = 6378137.0
    e2 = 6.69437999014e-3
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    N = a / np.sqrt(1 - e2 * np.sin(lat_rad)**2)
    x = (N + alt) * np.cos(lat_rad) * np.cos(lon_rad)
    y = (N + alt) * np.cos(lat_rad) * np.sin(lon_rad)
    z = (N * (1 - e2) + alt) * np.sin(lat_rad)
    return x, y, z

# ====== 설정 ======
use_gps_filtering = False  # 🚩 True: 필터링 사용, False: 전체 사용

target_gps_coords = np.array([
    [37.47045, 126.9582],
    [37.47051, 126.9593],
    [37.47242, 126.9594],
    [37.47242, 126.9583]
])
# ==================

target_ecef_coords = np.array([geodetic_to_ecef(lat, lon) for lat, lon in target_gps_coords])
transformer = ecef_to_utm_transformer(zone_number=52, zone_letter="north")
target_utm_coords = ecef_to_utm_points(target_ecef_coords, transformer)

# ====== 포인트 클라우드 준비 ======
points = np.asarray(pcd.points).astype(np.float64)
colors = np.asarray(pcd.colors) if pcd.has_colors() else None
normals = np.asarray(pcd.normals) if pcd.has_normals() else None

if use_gps_filtering:
    # (1) x, y 추출
    points_utm = ecef_to_utm_points(points, transformer)
    point_xy = points_utm[:, :2]

    # (2) 마스크 생성
    inside_ind = filter_points_in_polygon(point_xy, target_utm_coords)

    # (3) 필터링 적용
    inside_points = points[inside_ind]
    inside_colors = colors[inside_ind] if colors is not None else None
    inside_normals = normals[inside_ind] if normals is not None else None

    print(f"Filtered point numbers: {len(inside_points)}")
else:
    # 필터링 없이 전체 사용
    inside_points = points
    inside_colors = colors
    inside_normals = normals
    print(f"No filtering applied. Total points: {len(inside_points)}")


No filtering applied. Total points: 9998607


In [16]:
# 1. Height-based Filetering

# 높이가 0m 이상인 포인트만 필터링
filtered_indices = inside_points[:, 2] >= -100.0  # Z 값이 thr 이상인 인덱스를 찾음
filtered_points = inside_points[filtered_indices]  # Z 값이 thr 이상인 포인트들만 선택

# 3. 통계적으로 높이 범위 벗어나는 노이즈 제거
z_values = filtered_points[:, 2]  # Z 값만 추출
mean_z = np.mean(z_values)
std_z = np.std(z_values)

# 예시: 평균 ± 3 * 표준편차 범위 내의 포인트들만 남김
z_threshold_upper = mean_z + 3 * std_z
z_threshold_lower = mean_z - 1.5 * std_z

print(f"Filtered hegiht range: {z_threshold_lower} ~ {z_threshold_upper}")
# 범위 밖의 포인트 제거
final_filtered_indices = (z_values >= z_threshold_lower) & (z_values <= z_threshold_upper)
filtered_points = filtered_points[final_filtered_indices]

# 색상 및 법선 벡터도 필터링
if colors is not None:
    filtered_colors = inside_colors[filtered_indices][final_filtered_indices]
else:
    filtered_colors = None

if normals is not None:
    filtered_normals = inside_normals[filtered_indices][final_filtered_indices]
else:
    filtered_normals = None

filtered_pcd = o3d.geometry.PointCloud()
filtered_pcd.points = o3d.utility.Vector3dVector(filtered_points)
# 색상 및 법선 벡터도 필터링한 값으로 업데이트
if filtered_colors is not None:
    filtered_pcd.colors = o3d.utility.Vector3dVector(filtered_colors)

if filtered_normals is not None:
    filtered_pcd.normals = o3d.utility.Vector3dVector(filtered_normals)
    
# 필터링된 포인트 수 출력
print(f"Filtered point numbers: {len(filtered_points)}")

Filtered hegiht range: 48.9842511492743 ~ 108.69018882876587
Filtered point numbers: 9004462


In [17]:
# 2. SOR (Statistical Outlier Removal) 노이즈 제거
nb_neighbors = 20  # 각 포인트에 대해 고려할 이웃 포인트의 개수
std_ratio = 1.0    # 표준편차를 기준으로 이상치 제거 기준 (표준편차 배수)


# remove_statistical_outlier는 포인트 수와 인덱스를 반환
sor_pcd, sor_idx = filtered_pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
print(f"SOR Filtered point count: {len(sor_pcd.points)}")

# .ply 파일로 저장
o3d.io.write_point_cloud(output_ply_path, sor_pcd)
print(f"Point cloud saved to: {output_ply_path}")

SOR Filtered point count: 8179208
Point cloud saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/250526_STEAM_Naju_workspace/250526_earthwork_90m_correct_trial2/initial/nerf_output/exports/pcd/filtered/filtered_utm.ply


In [ ]:
# EXT-3. grid-based downsample
import numpy as np

def grid_downsample_mean_z_with_attributes(points, colors=None, normals=None, grid_size=0.01):
    """
    포인트 클라우드를 그리드 단위로 다운샘플링하고, z 평균으로 하나의 점을 남긴다.
    colors와 normals도 각각 평균으로 병합 (선택사항)

    Returns:
        down_points: (M, 3)
        down_colors: (M, 3) or None
        down_normals: (M, 3) or None
    """
    xy_indices = np.floor(points[:, :2] / grid_size).astype(np.int32)
    keys = [tuple(idx) for idx in xy_indices]

    grid_dict = {}
    for i, key in enumerate(keys):
        if key not in grid_dict:
            grid_dict[key] = {
                "z_list": [],
                "color_list": [],
                "normal_list": []
            }
        grid_dict[key]["z_list"].append(points[i, 2])
        if colors is not None:
            grid_dict[key]["color_list"].append(colors[i])
        if normals is not None:
            grid_dict[key]["normal_list"].append(normals[i])

    down_points, down_colors, down_normals = [], [], []
    for key, val in grid_dict.items():
        key = np.array(key, dtype=np.float64)
        x_center = (key[0] + 0.5) * grid_size
        y_center = (key[1] + 0.5) * grid_size        
        z_mean = np.mean(val["z_list"])
        down_points.append([x_center, y_center, z_mean])

        if colors is not None:
            down_colors.append(np.mean(val["color_list"], axis=0))
        if normals is not None:
            down_normals.append(np.mean(val["normal_list"], axis=0))

    return (
        np.array(down_points),
        np.array(down_colors) if colors is not None else None,
        np.array(down_normals) if normals is not None else None
    )

sor_points = np.asarray(sor_pcd.points).astype(np.float64)
sor_colors = filtered_colors[sor_idx]
sor_normals = filtered_normals[sor_idx]
down_points, down_colors, down_normals = grid_downsample_mean_z_with_attributes(sor_points, sor_colors, sor_normals, grid_size=0.05)

# 4. 필터링된 포인트 클라우드로 업데이트
down_pcd = o3d.geometry.PointCloud()
down_pcd.points = o3d.utility.Vector3dVector(down_points)

# 색상 및 법선 벡터도 필터링한 값으로 업데이트
if down_colors is not None:
    down_pcd.colors = o3d.utility.Vector3dVector(down_colors)

if down_normals is not None:
    down_pcd.normals = o3d.utility.Vector3dVector(down_normals)

output_ply_path = os.path.join(workspace_dir,'nerf_output/exports/pcd/filtered/grid_filtered_utm.ply')

# .ply 파일로 저장
o3d.io.write_point_cloud(output_ply_path, down_pcd)

print("원래 포인트 수:", len(sor_points))
print("다운샘플 후:", down_points.shape[0])

원래 포인트 수: 6942341
다운샘플 후: 3561756


In [7]:
# 3. interpolate color
import scipy


def patch_interpolate_color(pcd):

    # 1. 포인트 클라우드의 numpy 배열로 변환
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else None

    trunc_fun = lambda x: np.round(x) # ex. 1m precision

    # 3. 평면 상에서 보간할 그리드 정의
    x_min, y_min = trunc_fun(np.min(points[:, 0])), trunc_fun(np.min(points[:, 1]))
    x_max, y_max = trunc_fun(np.max(points[:, 0])), trunc_fun(np.max(points[:, 1]))
    
    print([x_min, y_min, x_max, y_max])

    # 그리드의 해상도 정의
    grid_x, grid_y = np.mgrid[x_min:x_max:0.05, y_min:y_max:0.05]  # 0.05m 간격의 그리드

    # 4. 보간을 위한 데이터 준비
    # 기존 포인트의 x, y 값
    xy_points = points[:, :2]

    # 6. Color 보간
    grid_colors = np.empty((grid_x.shape[0], grid_x.shape[1], 3), dtype=np.float64)
    for i in range(3):  # 각 채널(RGB)에 대해 보간
        grid_colors[:, :, i] = scipy.interpolate.griddata(xy_points, colors[:, i], (grid_x, grid_y), method='nearest')
    
    
    min_max = (x_min, y_min, x_max, y_max)
    return min_max, grid_colors #(num of grid points, 3)

min_max_color_phase, grid_colors = patch_interpolate_color(filtered_pcd)

[299877.0, 3874302.0, 300300.0, 3874898.0]


KeyboardInterrupt: 

In [31]:
# 4. 소수점 아래 2번째 자리까지 반올림 -> 격자화

# PointCloud의 점 데이터를 numpy 배열로 가져오기
points = np.asarray(sor_pcd.points)

# 소수점 아래 2자리까지만 남기고 나머지 제거
rounded_points = np.trunc(points * 100) * 0.01
print(rounded_points[:10, 1])
# 소수점 아래 2자리까지 반올림한 점들을 다시 PointCloud 객체에 설정
sor_pcd.points = o3d.utility.Vector3dVector(rounded_points)

# 결과를 새로운 .ply 파일로 저장
output_ply_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_utm_floor.ply"
o3d.io.write_point_cloud(output_ply_path, sor_pcd)

print(f"Point cloud with rounded coordinates saved to: {output_ply_path}")

[4147475.85 4147551.97 4147429.86 4147419.11 4147479.66 4147416.16
 4147493.76 4147413.16 4147520.45 4147401.08]
Point cloud with rounded coordinates saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_utm_floor.ply


In [32]:
# 5. patch_interpolate_z

def patch_interpolate_z(pcd, min_max_color_phase, grid_colors, step=0.05):

    # 1. 포인트 클라우드의 numpy 배열로 변환
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else None
    normals = np.asarray(pcd.normals) if pcd.has_normals() else None

    trunc_fun = lambda x: np.round(x) # ex. 1m precision
    # 3. 평면 상에서 보간할 그리드 정의
    x_min, y_min = trunc_fun(np.min(points[:, 0])), trunc_fun(np.min(points[:, 1]))
    x_max, y_max = trunc_fun(np.max(points[:, 0])), trunc_fun(np.max(points[:, 1]))

    # 그리드의 해상도 정의
    grid_x, grid_y = np.mgrid[x_min:x_max:step, y_min:y_max:step]  # 0.05m 간격의 그리드

    # 4. 보간을 위한 데이터 준비
    # 기존 포인트의 x, y 값
    xy_points = points[:, :2]

    # z 값은 모두 0, 주변값을 기준으로 보간할 예정
    z_values = points[:, 2]

    # 5. 보간: griddata 사용하여 빈공간을 채우기
    # 방법: 'linear', 'cubic', 'nearest' 등을 사용할 수 있음
    grid_z = scipy.interpolate.griddata(xy_points, z_values, (grid_x, grid_y), method='nearest')

    # 6. Color 보간
    c_x_min, c_y_min, c_x_max, c_y_max = min_max_color_phase
    x_min_ind = int(trunc_fun((x_min - c_x_min)/step))
    y_min_ind = int(trunc_fun((y_min - c_y_min)/step))
    x_max_ind = grid_colors.shape[0] - int(trunc_fun((c_x_max - x_max)/step))
    y_max_ind = grid_colors.shape[1] - int(trunc_fun((c_y_max - y_max)/step))
    grid_colors = grid_colors[x_min_ind:x_max_ind, y_min_ind:y_max_ind]
    

    
    # 7. Normal 보간
    grid_normals = np.empty((grid_x.shape[0], grid_x.shape[1], 3), dtype=np.float64)
    for i in range(3):  # 각 노멀 벡터 성분에 대해 보간
        grid_normals[:, :, i] = scipy.interpolate.griddata(xy_points, normals[:, i], (grid_x, grid_y), method='nearest')

    # 보간 후, 노멀 벡터를 단위 벡터로 정규화
    norm_magnitudes = np.linalg.norm(grid_normals, axis=2, keepdims=True)
    grid_normals /= norm_magnitudes  # 정규화

    # 8. 보간된 결과를 3D 포인트 클라우드로 변환
    interpolated_points = np.vstack((grid_x.flatten(), grid_y.flatten(), grid_z.flatten())).T
    interpolated_colors = grid_colors.reshape(-1, 3)
    interpolated_normals = grid_normals.reshape(-1, 3)

    # 9. Open3D 포인트 클라우드 객체로 변환
    interpolated_pcd = o3d.geometry.PointCloud()
    interpolated_pcd.points = o3d.utility.Vector3dVector(interpolated_points)
    interpolated_pcd.colors = o3d.utility.Vector3dVector(interpolated_colors)
    interpolated_pcd.normals = o3d.utility.Vector3dVector(interpolated_normals)

    return interpolated_pcd

interpolated_pcd = patch_interpolate_z(sor_pcd, min_max_color_phase, grid_colors)

# 결과를 새로운 .ply 파일로 저장
output_ply_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_interpolate_utm.ply"
o3d.io.write_point_cloud(output_ply_path, interpolated_pcd)

print(f"Point cloud saved to: {output_ply_path}")

318799.0 4147372.0 318950.0 4147595.0
0.0
100.0
60.0
200.0
(3020, 4460)
(3020, 4460, 3)


/tmp/ipykernel_13887/607795458.py:52: RuntimeWarning: invalid value encountered in divide
  grid_normals /= norm_magnitudes  # 정규화


Point cloud saved to: /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_interpolate_utm.ply


In [24]:
# 4. Patch-based Plane Interpolation
import open3d as o3d
import numpy as np

# 포인트 클라우드를 X, Y 기준으로 나누기 위한 함수
def split_point_cloud(pcd, x_step=1.0, y_step=1.0):
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else None
    normals = np.asarray(pcd.normals) if pcd.has_normals() else None

    x_min, y_min = np.min(points[:, 0]), np.min(points[:, 1])
    x_max, y_max = np.max(points[:, 0]), np.max(points[:, 1])

    # X, Y 범위에 맞게 그리드 정의
    x_range = np.arange(x_min, x_max, x_step)
    y_range = np.arange(y_min, y_max, y_step)

    # 그리드 상에 포인트 클라우드를 분할
    sub_clouds = []
    for x_start in x_range:
        for y_start in y_range:
            # 현재 그리드에 해당하는 점들을 필터링
            mask = (points[:, 0] >= x_start) & (points[:, 0] < x_start + x_step) & \
                   (points[:, 1] >= y_start) & (points[:, 1] < y_start + y_step)

            sub_points = points[mask]            
            sub_colors = colors[mask]
            sub_normals = normals[mask]

            if len(sub_points) > 0:  # 포인트가 존재하는 경우
                sub_pcd = o3d.geometry.PointCloud()
                sub_pcd.points = o3d.utility.Vector3dVector(sub_points)
                sub_pcd.colors = o3d.utility.Vector3dVector(sub_colors)
                sub_pcd.normals = o3d.utility.Vector3dVector(sub_normals)
                sub_clouds.append(sub_pcd)

    return sub_clouds



# 예시 포인트 클라우드 로드 (sor_pcd는 이미 정리된 포인트 클라우드)

input_ply_path = '/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_utm.ply'
pcd = o3d.io.read_point_cloud(input_ply_path)

# 포인트 클라우드를 x, y 기준으로 나누기
splited_sub_clouds = split_point_cloud(pcd, x_step=5, y_step=5)  # 원하는 크기만큼 잘라서 처리

In [30]:
# plane 추출
import open3d as o3d
import numpy as np
import scipy.interpolate

def sub_pointcloud_interpolation(sub_clouds, inlier_thr = 0.8):
    cnt = 0
    max_ratio = 0.0
    interpolated_sub_clouds = []
    for pcd in sub_clouds:
        if(len(pcd.points) <3):
            # print("too small pcd")
            continue
        plane_model, inliers = pcd.segment_plane(distance_threshold=0.10,
                                                ransac_n=3,
                                                num_iterations=1000)
        [a, b, c, d] = plane_model
        # print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
        
        total = len(pcd.points)
        inlier_ratio = len(inliers)/ total
        if(max_ratio < inlier_ratio):
            max_ratio = inlier_ratio

        if(inlier_ratio >= inlier_thr):
            cnt += 1        
            inlier_pcd = pcd.select_by_index(inliers)
            interpolated_pcd = patch_interpolate(inlier_pcd)
            interpolated_sub_clouds.append(interpolated_pcd)

    print(cnt)
    print(max_ratio)
    points = np.empty((0, 3))
    colors = np.empty((0, 3))
    normals = np.empty((0, 3))
    for sub_pcd in interpolated_sub_clouds:
        points = np.vstack((points, sub_pcd.points))
        colors = np.vstack((colors, sub_pcd.colors))
        normals = np.vstack((normals, sub_pcd.normals))

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    pcd.normals = o3d.utility.Vector3dVector(normals)

    return pcd

# .ply 파일로 저장
inlier_thr = 0.4
pcd = sub_pointcloud_interpolation(splited_sub_clouds, inlier_thr)
output_ply_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/test/test.ply"
o3d.io.write_point_cloud(output_ply_path, pcd)

/tmp/ipykernel_94030/910921686.py:38: RuntimeWarning: invalid value encountered in divide
  grid_normals /= norm_magnitudes  # 정규화


139
1.0


True

In [ ]:

mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
pcd, depth=9)

output_ply_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/test/test.ply"
o3d.io.write_triangle_mesh(output_ply_path, mesh)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Loaded 5757153 points from /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_utm.ply


[WARNING] /root/Open3D/build/poisson/src/ext_poisson/PoissonRecon/Src/FEMTree.IsoSurface.specialized.inl (Line 1858)
          Extract
          bad average roots: 3


True

In [ ]:
import open3d as o3d

input_ply_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_utm.ply"

pcd = o3d.io.read_point_cloud(input_ply_path)
print(f"Loaded {len(pcd.points)} points from {input_ply_path}")

pcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

output_ply_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/test/test.ply"
o3d.io.write_point_cloud(output_ply_path, pcd)

Loaded 5757153 points from /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_utm.ply


True

In [6]:
# ECEF to UTM
from pyproj import Transformer
import numpy as np
import open3d as o3d
def utm_to_ecef_transformer(zone_number: int, zone_letter: str):
    """Creates a Transformer for UTM to ECEF conversion."""
    return Transformer.from_crs(f"+proj=utm +zone={zone_number} +{zone_letter} +datum=WGS84", "EPSG:4978")

def utm_to_ecef_points(points_utm: np.ndarray, transformer: Transformer) -> np.ndarray:
    """
    Converts UTM points to ECEF coordinates.
    
    points_utm : (N, 3) - UTM coordinates (E, N, Altitude)
    """
    
    e, n, alt = points_utm[:, 0], points_utm[:, 1], points_utm[:, 2]
    x, y, z = transformer.transform(e, n, alt)
    points_ecef = np.stack([x, y, z], axis=-1)
    
    return np.array(points_ecef)

input_ply_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_utm.ply"
output_ply_path = "/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_ecef.ply"

pcd = o3d.io.read_point_cloud(input_ply_path)
coords_utm = np.asarray(pcd.points)
print(f"Loaded {len(coords_utm)} points from {input_ply_path}")

# Convert ECEF to UTM
transformer = utm_to_ecef_transformer(zone_number=52, zone_letter="north")
coords_ecef = utm_to_ecef_points(coords_utm, transformer)

pcd.points = o3d.utility.Vector3dVector(coords_ecef)
o3d.io.write_point_cloud(output_ply_path, pcd)

Loaded 7724974 points from /workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/SNU-35Back_workspace/50m/exports/pcd/filtered_utm.ply


True

In [ ]:
from pyproj import Transformer
import numpy as np
import open3d as o3d
def utm_to_ecef_transformer(zone_number: int, zone_letter: str):
    """Creates a Transformer for UTM to ECEF conversion."""
    return Transformer.from_crs(f"+proj=utm +zone={zone_number} +{zone_letter} +datum=WGS84", "EPSG:4978")

def utm_to_ecef_points(points_utm: np.ndarray, transformer: Transformer) -> np.ndarray:
    """
    Converts UTM points to ECEF coordinates.
    
    points_utm : (N, 3) - UTM coordinates (E, N, Altitude)
    """
    
    e, n, alt = points_utm[:, 0], points_utm[:, 1], points_utm[:, 2]
    x, y, z = transformer.transform(e, n, alt)
    points_ecef = np.stack([x, y, z], axis=-1)
    
    return np.array(points_ecef)

coords_utm_y1 = np.array([    
    [318890.442133, 4147487.223265, 140.161864],
        [318890.909859, 4147481.202181, 140.201193],
        [318895.667564, 4147481.597686, 140.240356], 
        [318895.201717, 4147487.595746, 140.205322],    
    ], dtype=np.float64)

coords_utm_j1 = np.array([    
    [318890.849485, 4147481.063982, 140.179913],
        [318891.481113, 4147475.233882, 140.122883],
        [318896.339060, 4147475.711492, 140.231966], 
        [318895.749198, 4147481.881790, 140.243442],    
    ], dtype=np.float64)

transformer = utm_to_ecef_transformer(zone_number=52, zone_letter="north")
coords_ecef_y1 = utm_to_ecef_points(coords_utm_y1, transformer)
coords_ecef_j1 = utm_to_ecef_points(coords_utm_j1, transformer)
print(coords_ecef_j1)

[[-3047504.68074928  4051184.25442885  3857815.59610601]
 [-3047507.34951665  4051186.55475769  3857811.00875926]
 [-3047511.06631932  4051183.40606937  3857811.55604814]
 [-3047508.20908455  4051180.82131159  3857816.5130461 ]]


In [2]:
from pyproj import Transformer
import numpy as np
import open3d as o3d

def ecef_to_utm_transformer(zone_number: int, zone_letter: str):
    """Creates a Transformer for ECEF to UTM conversion."""
    return Transformer.from_crs("EPSG:4978", f"+proj=utm +zone={zone_number} +{zone_letter} +datum=WGS84")

def ecef_to_utm_points(points_ecef: np.ndarray , transformer: Transformer) -> np.ndarray:
    """
    Converts ECEF points to UTM coordinates.
    
    points_ecef : (N, 3)
    """

    x, y, z = points_ecef[:, 0], points_ecef[:, 1], points_ecef[:, 2]
    e, n, alt = transformer.transform(x, y, z)
    points_utm = np.stack([e, n, alt], axis = -1) 
    
    return np.array(points_utm)

trans_cloudcompare = np.array([3047000.00, -4051000.00, -3857000.00], dtype = np.float64)


coords_ecef_j1 = np.array(
    [
        [-506.078119, 182.807889, 816.023397],
        [-507.263343, 186.535461, 811.139028],         
        [-508.224556, 184.170274, 812.938215], # 소의 ㅗ
        [-510.949760, 183.534916, 811.549471],
        ]
) 

coords_ecef_j1 -= trans_cloudcompare

coords_ecef_y1 = np.array(
        [[-505.581044, 178.509305, 820.971791],
        [-501.903964, 181.685998, 820.466010],
        [-504.697447, 181.360312, 818.597099],
        [-505.958553, 182.689592, 816.230171]]
    )    

coords_ecef_y1 -= trans_cloudcompare


   

# Convert ECEF to UTM
transformer = ecef_to_utm_transformer(zone_number=52, zone_letter="north")
coords_utm_j1 = ecef_to_utm_points(coords_ecef_j1, transformer)
coords_utm_y1 = ecef_to_utm_points(coords_ecef_y1, transformer)
print(coords_utm_j1[:, :2])
print("")
print(coords_utm_y1[:, :2])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[[ 318892.84107189 4147481.48802571]
 [ 318891.41463701 4147475.39550037]
 [ 318893.65244937 4147477.57361983]
 [ 318896.17264202 4147475.72848789]]

[[ 318895.16213698 4147487.63786774]
 [ 318890.30214663 4147487.14235971]
 [ 318892.67867724 4147484.74360234]
 [ 318892.82241112 4147481.75383504]]


In [ ]:
import numpy as np
# Final evaluation(ATE)
trans_cloudcompare = np.array([3047000.00, -4051000.00, -3857000.00], dtype = np.float64)
GT_feature_vertices = np.array([
        [-501.967988, 181.675080, 820.428803],
        [-507.307352, 186.436744, 811.205693],
        [-510.972234, 183.477112, 811.603174],
        [-505.591991, 178.600039, 820.896150]
    ])

GT_feature_vertices -= trans_cloudcompare
    
    
    
# .ply 파일 읽기        
input_ply_path = "/workspace/Laboratory/00.CourseWork/24-2_ITAutocon/00.workspace/CCTV_result/model_based_tracking.ply"
pcd = o3d.io.read_point_cloud(input_ply_path)
print(f"Loaded {len(pcd.points)} points from {input_ply_path}")

pred_points = np.asarray(pcd.points)  
print(len(pred_points))


GT_feature_vertices = ecef_to_utm_points(GT_feature_vertices, transformer)
pred_points = ecef_to_utm_points(pred_points, transformer)

GT_points_filename = "/workspace/Laboratory/00.CourseWork/24-2_ITAutocon/00.workspace/CCTV_result/GT.npy"
np.save(GT_points_filename, GT_feature_vertices)
print(f"Predicted points saved to {GT_points_filename}")

pred_points_filename = "/workspace/Laboratory/00.CourseWork/24-2_ITAutocon/00.workspace/CCTV_result/pred_points_uav.npy"
np.save(pred_points_filename, pred_points)
print(f"Predicted points saved to {pred_points_filename}")


Loaded 188 points from /workspace/Laboratory/00.CourseWork/24-2_ITAutocon/00.workspace/CCTV_result/model_based_tracking.ply
188
Predicted points saved to /workspace/Laboratory/00.CourseWork/24-2_ITAutocon/00.workspace/CCTV_result/GT.npy
Predicted points saved to /workspace/Laboratory/00.CourseWork/24-2_ITAutocon/00.workspace/CCTV_result/pred_points_uav.npy


In [3]:
# PCD centering with translation.json

import open3d as o3d
import json
import numpy as np

# 1. PCD 파일 불러오기
pcd = o3d.io.read_point_cloud("/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/exports/pcd/filtered/filtered_ck.ply")  # 'input.pcd'는 실제 파일 경로로 변경

# 2. JSON 파일에서 translation 값 읽기
with open("/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/exports/3dmesh/gps/translation_ck.json", "r") as f:
    json_data = json.load(f)
translation_data = json_data["global_translation"]

# 3. x, y, z 추출


# 4. 포인트 클라우드에 translation 적용
translation_vector = - np.array(translation_data)
pcd.translate(translation_vector)

# 5. 결과 저장
o3d.io.write_point_cloud("/workspace/Laboratory/02.Rapid3DReconstruction/00.workspace/Nakseongdae_50m_oblique_workspace/trial1/initial/nerf_output/exports/pcd/filtered/centered_filtered_ck.pcd", pcd)


True